<a href="https://colab.research.google.com/github/samsol38/numpy_v1/blob/main/numpy_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import contextlib
import io

@contextlib.contextmanager
def suppress_output():
    with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
        yield

with suppress_output():
  %pip install matrepr solara "solara[assets]"

In [128]:
import numpy as np
import solara
from matrepr import mdisplay, mprint, to_html, to_str, to_latex
from IPython.display import display, Math, Latex
import matrepr
np.random.seed(1000)

In [137]:
state = solara.reactive({
    "xlabel": "XPoints",
    "ylabel": "YPoints",
    "grid": False,
    "rotation": 0})

master_box_style = {"border-style": "dashed",
      "border-width": '2px',
      "border-color": '#000',
      "border-radius": "8px",
      "flex-shrink": 1,
      "flex-wrap": 'no-wrap'}

box_style = {"border-style": "dashed",
      "border-width": '1px',
      "border-color": '#000',
      "border-radius": "8px"}

array_math_funcs = [
      'np.exp(array)',
      'np.sqrt(array)',
      'np.sin(array)',
      'np.cos(array)',
      'np.log((array))',
      'np.transpose(array)',
      'array.ravel()',
      'array.flatten()'
]

array_math_func = solara.reactive(array_math_funcs[0])

class NumPyArrayFuncViewer:
  def __init__(self, array):
    self.array = array
    self.showArray()

  def showArray(self):
    with solara.Row():
      with solara.Card(elevation=0, style=box_style):
        vectorized_format = np.vectorize(lambda x: float(f"{x:.2f}"))
        solara.Select(label="Select function to calculate array", value=array_math_func, values=array_math_funcs)
        solara.Markdown('`{} = `'.format(array_math_func.value))
        solara.display(Math(r'{}'.format(to_latex(vectorized_format(eval(array_math_func.value.replace('array',
                                                                                     'np.array({})'.format(str(self.array.tolist())))).tolist())))))

class NumPyViewer:

  def __init__(self, array):
    self.array = array
    self.showArray()

  def get_rounded_value(self, value):
    return round(value, 2)

  def gen_field_report(self, tupple):
    return "`{} =` **{}**</br>".format(tupple[0], tupple[1])

  def gen_report(self, fieldList):
    markdown_report = """
    {}
    """.format("".join(list(map(lambda tupple: self.gen_field_report(tupple), fieldList))))
    return markdown_report

  def gen_attr_report(self):
    markdown_report = self.gen_report([
      ('array.shape()', self.array.shape),
      ('len(array)', len(self.array)),
      ('array.ndim', self.array.ndim),
      ('array.size', self.array.size),
      ('array.dtype', self.array.dtype),
      ('array.dtype.name', self.array.dtype.name)])
    return markdown_report

  def gen_stats_report(self):
    markdown_report = self.gen_report([
      ('array.sum()', self.array.sum()),
      ('array.min()', self.array.min()),
      ('array.max()', self.array.max()),
      ('array.mean()', self.get_rounded_value(self.array.mean())),
      ('np.median(array)', np.median(self.array)),
      ('np.std(array)', self.get_rounded_value(np.std(self.array)))])
    return markdown_report

  def showArray(self):
    with solara.Row():
      with solara.Card(elevation=0, style=box_style):
        solara.Markdown('`array = `')
        solara.display((Math(r'{}'.format(to_latex(self.array)))))
      with solara.lab.Tabs():
          with solara.lab.Tab(label="Attributes"):
            with solara.Card(elevation=0, style=box_style):
              solara.Markdown(self.gen_attr_report())
          with solara.lab.Tab(label="Statistics"):
            with solara.Card(elevation=0, style=box_style):
              solara.Markdown(self.gen_stats_report())
          with solara.lab.Tab(label="Manipulation"):
            _ = NumPyArrayFuncViewer(self.array)

class NumPyPage:
  def __init__(self, title, array):
    self.render_page(title, array)

  def render_page(self, title, array):
    with solara.Card(title=title, elevation=0, style=master_box_style):
      solara.Select(label="Select function to create array", value=create_array_func, values=create_array_funcs)
      solara.Markdown("`import numpy as np`</br>`array = {}`".format(array))
      with solara.Row():
        with solara.HBox():
          NumPyViewer(eval(array))

create_array_funcs = [
      'np.array([[32, 54, 23, 65], [44, 36, 73, 54], [53, 64, 72, 85]])',
      'np.random.randint(low=11, high=30, size=(3, 5), dtype=np.uint8)',
      'np.full((2, 2), 7)',
      'np.eye(4)',
      'np.zeros((3, 4))',
      'np.ones((5, 3, 4))',
      'np.arange(10, 40, 5)',
      'np.linspace(0, 2, 9)',
      'np.empty((3, 2))'
]

create_array_func = solara.reactive(create_array_funcs[0])

@solara.component
def MainComponent():

  _ = NumPyPage("Creating Array", create_array_func.value)

MainComponent()


Html(layout=None, style_='display: none', tag='span')

Cannot show ipywidgets in text